In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import random

In [9]:
USER_AGENT_LIST = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36']
DRIVER_FILE_PATH = "/Users/qunishdash/Documents/chromedriver_mac64/chromedriver"

In [10]:
def get_chrome_driver(headless_flag):
    # Set up the Selenium webdriver
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service

    service = Service(executable_path=DRIVER_FILE_PATH)
    chrome_options = Options()
    
    if headless_flag:
        # in case you want headless browser  
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("user-agent={}".format(random.choice(USER_AGENT_LIST)))
    
    driver = webdriver.Chrome(service=service, options=chrome_options) 

    return driver

In [11]:
driver = get_chrome_driver(False)

In [12]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["walmart_data_db"]
collection = db["dorm_bedding_LV"]

In [13]:
from selenium.webdriver.common.by import By
def get_product_data(url):

    driver.get(url)
    try:
        product_details = driver.find_element(By.CSS_SELECTOR, '.nb3').text
    except Exception as e:
        product_details = ''
    try:
        product_specification = driver.find_element(By.CSS_SELECTOR, '.ph3:nth-child(6) .pt1').text
    except Exception as e:
        product_specification = ''
    try:
        product_instruction = driver.find_element(By.CSS_SELECTOR, '.ph3:nth-child(8) .pt1').text
    except Exception as e:
        product_instruction = ''
    try:
        product_warranty = driver.find_element(By.CSS_SELECTOR, '.ph3:nth-child(10) .pt1').text
    except Exception as e:
        product_warranty = ''

    # Create a dictionary with the scraped data
    product_data = {
        "product_details": product_details,
        "product_specification": product_specification,
        "product_instruction": product_instruction,
        "product_warranty": product_warranty
    }
    
    return product_data

In [14]:
import pandas as pd

if __name__ == "__main__":
    
    # Fetch product URLs from MongoDB
    data = collection.find({})
    data_result = list(data)
    df = pd.DataFrame(data_result)
    pv_urls =  df['canonicalUrl'].apply(lambda x: 'https://www.walmart.com' + x if pd.notna(x) else '')
    
    # Define a list to store scraped data
    scraped_data_list = []

    # Scrape data for each URL and store it in the 'scraped_data_list'
    for url in pv_urls:
        data = get_product_data(url)
        print(data
        scraped_data_list.append(data)

    # Upsert the data into MongoDB based on the 'onClickLink' URLs:
    try:
        for data in scraped_data_list:
            pv_collection = "dorm_bedding_PV"
            pvcollection = db[pv_collection]
            url = data["product_url"]  # Fetch the URL from the correct location in the data dictionary
            del data["product_url"]  # Remove the 'product_url' key as it's not part of the update
            pvcollection.update_one({"product_url": url}, {"$set": data}, upsert=True)
        print("Data upserted into MongoDB successfully.")
    except Exception as e:
        print("Error while upserting data into MongoDB:", e)

SyntaxError: invalid syntax (2988801104.py, line 18)

In [ ]:
client.close()

In [7]:
print(scraped_data_list[:2])

NameError: name 'scraped_data_list' is not defined

In [ ]:
driver.quit()